# Защита персональных данных клиентов

## Описание проекта

    Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.
   
    Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.
   
    Набор данных находится в файле /datasets/insurance.csv. Скачать датасет.
    
    Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
    Целевой признак: количество страховых выплат клиенту за последние 5 лет.

## 1. Загрузите и изучите данные.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings("ignore")
from IPython.display import display, Math, Latex

In [2]:
try:
    df = pd.read_csv('insurance.csv', sep=',')
except:
    df = pd.read_csv('/datasets/insurance.csv', sep=',')

Изучим данные.

In [3]:
print('Явные дубликаты:', df.duplicated().sum())
print('========================================')
print('Пропуски:')
print(df.isna().sum())
print('==============================================')
print('Общая информация:')
print(df.info())
print('=============================================================================')
print(df.describe())

Явные дубликаты: 153
Пропуски:
Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64
Общая информация:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None
               Пол      Возраст      Зарплата  Члены семьи  Страховые выплаты
count  5000.000000  5000.000000   5000.000000  5000.000000        5000.000000
mean      0.499000    30.952800  39916.360000     1.194200           0.148000
std       0.500049     8.440807   9900.083569     1.091387           0.463183
min       0.

In [4]:
df.head(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Выводы: 
    
    1. В данных нет пропусков
    2. Представленные значения в числовом формате
    3. Признаки состоят из 4 столбцов и 5000 строк
    4. Обнаруживаются явные дубликаты. Не претендую на истину, но думаю, что это могут быть совпадения, а не реальные дубликаты, поэтому оставлю как есть.
    5. Выбросов не замечено.

## 2. Ответьте на вопрос и обоснуйте решение. 
 Признаки умножают на обратимую матрицу P. Изменится ли качество линейной регрессии? 

Для выяснения вопроса воспользуемся свойствами матриц:
1. Умножение исходной матрицы на обратную дает нам единичную матрицу:

2. Произведение любой матрицы и единичной матрицы подходящего размера равно самой матрице:


3. Свойство обратной матрицы:  

4. Ассоциативность:

5. Транспонированное произведение матриц равно произведению транспонированных матриц, взятых в обратном порядке:


Запишем формулы в исходном и преобразованном виде:

In [5]:
# Модель с исходными данными
display(Math(r"a = Xw = X (X^TX)^{-1} X^T y"))
# Модельс преобразованными данными
display(Math(r"a' = (XP)w'"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Последовательно упростим выражение с помощью свойств выше:

In [6]:
display(Math(r"w' = ((XP)^TXP)^{-1} (XP)^T y"))
display(Math(r"a' = XP((XP)^TXP)^{-1} (XP)^T y = XP(P^T (X^T X) P)^{-1}(XP)^T y = XP P^{-1} (X^TX)^{-1} (P^T)^{-1}"))
display(Math(r"a' = XP P^{-1} (X^TX)^{-1} (P^T)^{-1}P^T X^T y = XE (X^TX)^{-1} E X^T y"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Используя свойтсво: "Произведение любой матрицы и единичной матрицы подходящего размера равно самой матрице", приводим выражение к виду:

In [7]:
display(Math(r"a' = X (X^TX)^{-1} X^T y"))

<IPython.core.display.Math object>

Таким образом, получилось равенство:

In [8]:
display(Math(r"a = a'"))

<IPython.core.display.Math object>

ВЫВОД: Качество линейной регрессии при умножении признаков на обратимую матрицу не изменится.

Исходный вариант   
$$
w = \arg\min_w MSE(Xw, y)
$$
    
  
и пребразованный    
    
$$
w_P = \arg\min_w MSE(XPw_p, y)
$$  
    
    
И делая  замену    w_p, видим что задача  
$$
\arg\min_w MSE(XPw_p, y)
$$

$$
w_p = P^{-1}w
$$ 

$$
\arg\min_w MSE(XPP^{-1}w, y)
$$

$$
\arg\min_w MSE(XEw, y)
$$

    
сводится к     
  
    
$$
\arg\min_w MSE(Xw, y)
$$

In [9]:
# display(Math(r"w' = ((XP)^TXP)^{-1} (XP)^T y = (P^T(X^TX)P)^{-1} P^TX^T y = P^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty "))
display(Math(r"w' = P^{-1}(X^TX)^{-1}EX^Ty  = P^{-1}(X^TX)^{-1}X^Ty"))
display(Math(r"w' =  P^{-1}w"))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

## 3. Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.

1. Создадим рандомную квадртантую матрицу 4х4
2. Проверим на обратимость
3. Если матрица не прошла проверку на обратимость - возвращаемся в п.1
4. Перемножим признаки и обратную матрицу.
5. Определим R2 на исходных и преобразованных данных
6. Сравним полученные значения

## 4. Запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.

Обучим модель и применим метрику R2 на исходных данных.

In [10]:
features = df.drop(['Страховые выплаты'],axis=1)
target = df['Страховые выплаты']
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

model = LinearRegression()
model.fit(features_train, target_train)
print("R2_first =", r2_score(target_test, model.predict(features_test)))

R2_first = 0.43522757127026546


R2 для исходных данных равна 0,4352275

Преобразуем признаки с помощью обратимой матрицы. 

In [11]:
# Исходный вид
features

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0
...,...,...,...,...
4995,0,28.0,35700.0,2
4996,0,34.0,52400.0,1
4997,0,20.0,33900.0,2
4998,1,22.0,32700.0,3


In [12]:
# создадим квадратную матрицу из случайных чисел
#p = np.random.rand(4,4)
# проверим на обратимость
#np.linalg.inv(p)

In [13]:
def get_rand_matrix(a):
    det = 0
    while det == 0:
        matrix = np.random.normal(size=(a, a))
        det = np.linalg.det(matrix)
    return matrix
get_rand_matrix(4)

array([[ 2.00875304, -0.82458401,  0.43624959, -0.73208335],
       [-0.96843717, -0.30118154,  0.19663429,  0.21549753],
       [-0.48345097, -1.28825213, -0.55127793, -1.56655868],
       [-1.16614303,  1.20438141,  1.96017336, -0.82054774]])

In [14]:
# перемножением преобразуем признаки
features_p = features@get_rand_matrix(4)
# Выведем результат преобразования
features_p

,0,1,2,3
0,-15328.953327,-36217.034915,201.855734,6646.505556
1,-11714.379127,-27735.800856,154.322503,5077.053052
2,-6464.994695,-15324.644522,85.178052,2801.973545
3,-12918.701143,-30459.485292,170.133023,5603.164820
4,-8051.356601,-19052.578256,106.010529,3490.083995
...,...,...,...,...
4995,-11039.586427,-26069.285120,145.400494,4786.033194
4996,-16214.214210,-38268.597733,213.556239,7032.966370
4997,-10496.978344,-24759.975843,138.242353,4551.840813
4998,-10121.639708,-23882.059472,133.241390,4386.473746


In [15]:
#Обучим модель и применим метрику R2 на преобразованных данных.
features_p_train, features_p_test, target_train, target_test = train_test_split(
    features_p, target, test_size=0.25, random_state=12345)

model = LinearRegression()
model.fit(features_p_train, target_train)
print("R2_second =", r2_score(target_test, model.predict(features_p_test)))


R2_second = 0.43522757127022393


R2 для преобразованных данных равна 0,4352275


ВЫВОД:
качество линейной регрессии не отличается до и после преобразования. Метрика R2 = 0,4352275.

    Данные были проверены на аномалии,пропуски и дубликаты.
    В целях защиты данных клиентов страховой компании был разработан метод преобразования данных: Признаки умножают на обратимую матрицу.
    В ходе работы было доказано, что данный метод не ухудшает качество модели машинного обучения после преобразования.
    Моделирование производилось с помощью LinearRegression.
    R2 для исходных данных равна 0,4352275
    R2 для преобразованных данных равна 0,4352275